### Check [this link](https://docs.microsoft.com/en-us/azure/storage/files/storage-python-how-to-use-file-storage?tabs=python) to connect python to Azure file share.

In [1]:
from azure.core.exceptions import (
    ResourceExistsError,
    ResourceNotFoundError
)

from azure.storage.fileshare import (
    ShareServiceClient,
    ShareClient,
    ShareDirectoryClient,
    ShareFileClient
)

#### You can find your storage account's connection strings in the Azure portal. Navigate to SETTINGS > Access keys in your storage account's menu blade to see connection strings for both primary and secondary access keys.

In [2]:
# Create a ShareServiceClient from a connection string
connection_string = '<here goes your connection string>'
service_client = ShareServiceClient.from_connection_string(connection_string)

## Create an Azure file share

In [8]:
def create_file_share(connection_string, share_name):
    try:
        # Create a ShareClient from a connection string
        share_client = ShareClient.from_connection_string(
            connection_string, share_name)

        print("Creating share:", share_name)
        share_client.create_share()

    except ResourceExistsError as ex:
        print("ResourceExistsError:", ex.message)

In [9]:
create_file_share(connection_string, 'test')

Creating share: test


In [7]:
share = ShareClient.from_connection_string(conn_str=connection_string, share_name="myshare")
share.create_share()

{'etag': '"0x8D8E8CE2EEA2B01"',
 'last_modified': datetime.datetime(2021, 3, 16, 22, 52, 32, tzinfo=datetime.timezone.utc),
 'request_id': 'f9a21a2a-501a-006d-79b7-1a4d33000000',
 'version': '2020-04-08',
 'date': datetime.datetime(2021, 3, 16, 22, 52, 31, tzinfo=datetime.timezone.utc)}

## Create a directory

In [15]:
def create_directory(connection_string, share_name, dir_name):
    try:
        # Create a ShareDirectoryClient from a connection string
        dir_client = ShareDirectoryClient.from_connection_string(
            connection_string, share_name, dir_name)

        print("Creating directory:", share_name + "/" + dir_name)
        dir_client.create_directory()

    except ResourceExistsError as ex:
        print("ResourceExistsError:", ex.message)

In [16]:
create_directory(connection_string, 'test', 'dict')

Creating directory: test/dict


## Upload file

The following method uploads **the contents** of the specified file into the specified directory in the specified Azure file share.

In [19]:
def upload_local_file(connection_string, local_file_path, share_name, dest_file_path):
    try:
        source_file = open(local_file_path, "rb")
        data = source_file.read()

        # Create a ShareFileClient from a connection string
        file_client = ShareFileClient.from_connection_string(
            connection_string, share_name, dest_file_path)

        print("Uploading to:", share_name + "/" + dest_file_path)
        file_client.upload_file(data)

    except ResourceExistsError as ex:
        print("ResourceExistsError:", ex.message)

    except ResourceNotFoundError as ex:
        print("ResourceNotFoundError:", ex.message)

In [23]:
local_file_path = r'<your local path>\logging\test.txt'
share_name = 'test'
dest_file_path = 'test_cloud.txt'
upload_local_file(connection_string, local_file_path, share_name, dest_file_path)

Uploading to: test/test_cloud.txt


## Enumerate files and directories in an Azure file share

In [24]:
def list_files_and_dirs(connection_string, share_name, dir_name):
    try:
        # Create a ShareClient from a connection string
        share_client = ShareClient.from_connection_string(
            connection_string, share_name)

        for item in list(share_client.list_directories_and_files(dir_name)):
            if item["is_directory"]:
                print("Directory:", item["name"])
            else:
                print("File:", dir_name + "/" + item["name"])

    except ResourceNotFoundError as ex:
        print("ResourceNotFoundError:", ex.message)

In [26]:
share_name = 'logs'
dir_name = 'dag_processor_manager'
list_files_and_dirs(connection_string, share_name, dir_name)

File: dag_processor_manager/dag_processor_manager.log
File: dag_processor_manager/dag_processor_manager.log.1
File: dag_processor_manager/dag_processor_manager.log.2
File: dag_processor_manager/dag_processor_manager.log.3
File: dag_processor_manager/dag_processor_manager.log.4
File: dag_processor_manager/dag_processor_manager.log.5


## Download a file

In [28]:
def download_azure_file(connection_string, share_name, dir_name, file_name):
    try:
        # Build the remote path
        source_file_path = dir_name + "/" + file_name

        # Add a prefix to the filename to 
        # distinguish it from the uploaded file
        dest_file_name = "DOWNLOADED-" + file_name

        # Create a ShareFileClient from a connection string
        file_client = ShareFileClient.from_connection_string(
            connection_string, share_name, source_file_path)

        print("Downloading to:", dest_file_name)

        # Open a file for writing bytes on the local system
        with open(dest_file_name, "wb") as data:
            # Download the file from Azure into a stream
            stream = file_client.download_file()
            # Write the stream to the local file
            data.write(stream.readall())

    except ResourceNotFoundError as ex:
        print("ResourceNotFoundError:", ex.message)

In [29]:
share_name = 'logs'
dir_name = 'dag_processor_manager'
file_name = 'dag_processor_manager.log'
download_azure_file(connection_string, share_name, dir_name, file_name)

### Delete a file

In [31]:
def delete_azure_file(connection_string, share_name, file_path):
    try:
        # Create a ShareFileClient from a connection string
        file_client = ShareFileClient.from_connection_string(
            connection_string, share_name, file_path)

        print("Deleting file:", share_name + "/" + file_path)

        # Delete the file
        file_client.delete_file()

    except ResourceNotFoundError as ex:
        print("ResourceNotFoundError:", ex.message)